In [197]:
from pybit import inverse_perpetual  # <-- import HTTP & WSS for inverse perp
from pybit import spot  # <-- import HTTP & WSS for spot
from pybit import HTTP  # supports inverse perp & futures, usdt perp, spot.
from pybit.spot import HTTP
from pprint import pprint
from dotenv import load_dotenv
import os
import json
import pandas as pd
from time import sleep
import ta 

In [198]:
load_dotenv()

True

In [199]:
api_key_pw = os.getenv('api_key_bot')
api_secret_pw = os.getenv('api_secret_bot')


In [200]:
session = inverse_perpetual.HTTP(
    endpoint='https://api.bybit.com', 
    api_key= api_key_pw,
    api_secret= api_secret_pw
)
ws = inverse_perpetual.WebSocket(
    test=False,
    api_key= api_key_pw,
    api_secret= api_secret_pw
)

In [201]:
pd.DataFrame(session.latest_information_for_symbol(symbol ='BTCUSD')["result"]).keys()

Index(['symbol', 'bid_price', 'ask_price', 'last_price', 'last_tick_direction',
       'prev_price_24h', 'price_24h_pcnt', 'high_price_24h', 'low_price_24h',
       'prev_price_1h', 'mark_price', 'index_price', 'open_interest',
       'countdown_hour', 'turnover_24h', 'volume_24h', 'funding_rate',
       'predicted_funding_rate', 'next_funding_time',
       'predicted_delivery_price', 'total_turnover', 'total_volume',
       'delivery_fee_rate', 'delivery_time', 'price_1h_pcnt', 'open_value'],
      dtype='object')

In [202]:
session.orderbook(symbol='BTCUSD')['result'][1]

{'symbol': 'BTCUSD', 'price': '16474', 'side': 'Buy', 'size': 71015}

In [203]:
from pybit import spot
session_unauth = spot.HTTP(
    endpoint="https://api-testnet.bybit.com"
)
session_unauth.public_trading_records(symbol="BTCUSDT")["result"]

[{'price': '16250',
  'time': 1669715406626,
  'qty': '0.003791',
  'isBuyerMaker': True,
  'type': '0'},
 {'price': '16250',
  'time': 1669715612099,
  'qty': '0.060611',
  'isBuyerMaker': True,
  'type': '0'},
 {'price': '16250',
  'time': 1669715612445,
  'qty': '0.060611',
  'isBuyerMaker': True,
  'type': '0'},
 {'price': '16250',
  'time': 1669715612741,
  'qty': '0.060611',
  'isBuyerMaker': True,
  'type': '0'},
 {'price': '16250',
  'time': 1669715613042,
  'qty': '0.060611',
  'isBuyerMaker': True,
  'type': '0'},
 {'price': '16250',
  'time': 1669715613339,
  'qty': '0.060612',
  'isBuyerMaker': True,
  'type': '0'},
 {'price': '16250',
  'time': 1669715613653,
  'qty': '0.060612',
  'isBuyerMaker': True,
  'type': '0'},
 {'price': '16251.2',
  'time': 1669715613957,
  'qty': '0.017133',
  'isBuyerMaker': True,
  'type': '0'},
 {'price': '16250',
  'time': 1669715613957,
  'qty': '0.029427',
  'isBuyerMaker': True,
  'type': '0'},
 {'price': '16250',
  'time': 1669715613957,

In [204]:
from pybit import spot
session_auth = spot.HTTP(
    endpoint='https://api.bybit.com',
    api_key= api_key_pw,
    api_secret= api_secret_pw
)
balance = pd.DataFrame(session_auth.get_wallet_balance()["result"])
balance

,balances
0,"{'coin': 'RUNE', 'coinId': 'RUNE', 'coinName':..."
1,"{'coin': 'USDC', 'coinId': 'USDC', 'coinName':..."
2,"{'coin': 'USDT', 'coinId': 'USDT', 'coinName':..."


In [205]:
def clean(dataframe):
    dataframe = balance.explode('balances').reset_index(drop= True)
    #dataframe = btc.explode("leverage_filter").reset_index(drop= True)
    dataframe = pd.concat([balance, balance["balances"].apply(pd.Series)], axis=1).reset_index(drop=True)
    #dataframe = pd.concat([btc, btc['leverage_filter'].apply(pd.Series)], axis=1).reset_index(drop=True)
    return dataframe

In [206]:
clean(balance)

,balances,coin,coinId,coinName,total,free,locked
0,"{'coin': 'RUNE', 'coinId': 'RUNE', 'coinName':...",RUNE,RUNE,RUNE,45.04,45.04,0
1,"{'coin': 'USDC', 'coinId': 'USDC', 'coinName':...",USDC,USDC,USDC,445.88,445.88,0
2,"{'coin': 'USDT', 'coinId': 'USDT', 'coinName':...",USDT,USDT,USDT,0.00569,0.00569,0


In [207]:
from pybit import spot
session_auth = spot.HTTP(
    endpoint='https://api.bybit.com',
    api_key = api_key_pw,
    api_secret= api_secret_pw
)
frame = pd.DataFrame(session_auth.query_kline(symbol="BTCUSDT", interval="1m")["result"])
frame

,0,1,2,3,4,5,6,7,8,9,10
0,1669657800000,16220.29,16221.27,16215,16217.95,1.209054,1669657860000,19608.28551873,51,0,0
1,1669657860000,16217.95,16224,16207.42,16222.56,10.844921,1669657920000,175862.47726529,180,0,0
2,1669657920000,16222.56,16222.56,16196,16201.25,4.287743,1669657980000,69480.7148828,210,0,0
3,1669657980000,16201.25,16212.75,16200.45,16209.16,8.600206,1669658040000,139372.94024833,128,0,0
4,1669658040000,16209.16,16215.78,16200,16202.23,2.12316,1669658100000,34412.91869081,64,0,0
...,...,...,...,...,...,...,...,...,...,...,...
995,1669717500000,16484.07,16490.94,16484.07,16487.17,0.816908,1669717560000,13469.38441031,27,0,0
996,1669717560000,16487.17,16492.42,16485.91,16488.85,0.196966,1669717620000,3247.49205525,37,0,0
997,1669717620000,16488.85,16498,16488.85,16498,4.250835,1669717680000,70114.72819777,44,0,0
998,1669717680000,16498,16498,16492,16492.71,2.043404,1669717740000,33700.53672755,22,0,0


In [278]:
def get15minutedata():
    frame = pd.DataFrame(session_auth.query_kline(symbol="BTCUSDT", interval="15m")["result"])
    frame = frame.iloc[:,: 6]
    frame.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    frame = frame.set_index("Time")
    frame.index = pd.to_datetime(frame.index, unit="ms")
    frame = frame.astype(float)
    return frame

In [279]:
df = get15minutedata()
df

,Open,High,Low,Close,Volume
Time,,,,,
2022-11-19 01:15:00,16665.19,16675.24,16656.33,16673.52,14.251999
2022-11-19 01:30:00,16673.52,16678.01,16650.32,16653.69,11.091925
2022-11-19 01:45:00,16653.69,16666.39,16651.45,16664.60,13.957144
2022-11-19 02:00:00,16664.60,16667.70,16607.72,16622.96,43.842921
2022-11-19 02:15:00,16622.96,16628.34,16584.58,16602.54,41.426830
...,...,...,...,...,...
2022-11-29 10:00:00,16513.63,16535.95,16488.49,16505.30,66.194366
2022-11-29 10:15:00,16505.30,16507.85,16480.52,16496.10,34.804686
2022-11-29 10:30:00,16496.10,16515.24,16491.08,16504.76,26.789706


In [286]:
def apply_technicals(df):
    df["K"] = ta.momentum.stochrsi(df.Close, window= 14, smooth1=3)
    #df["D"] = df["K"].rolling(3).mean()
    df["RSI"] = ta.momentum.rsi(df.Close, window = 14)
    df.dropna(inplace=True)


In [287]:
apply_technicals(df)

In [288]:
df

,Open,High,Low,Close,Volume,K,RSI
Time,,,,,,,
2022-11-19 18:45:00,16636.24,16636.24,16617.59,16623.21,9.054481,0.508620,39.451199
2022-11-19 19:00:00,16623.21,16636.85,16621.35,16627.86,7.400053,0.721192,42.647105
2022-11-19 19:15:00,16627.86,16633.64,16617.99,16629.88,17.839785,0.795633,44.029184
2022-11-19 19:30:00,16629.88,16633.96,16620.58,16625.65,7.708414,0.698204,41.759791
2022-11-19 19:45:00,16625.65,16630.47,16620.00,16628.59,10.529694,0.791084,43.923226
...,...,...,...,...,...,...,...
2022-11-29 10:00:00,16513.63,16535.95,16488.49,16505.30,66.194366,0.386760,63.330790
2022-11-29 10:15:00,16505.30,16507.85,16480.52,16496.10,34.804686,0.220559,61.004395
2022-11-29 10:30:00,16496.10,16515.24,16491.08,16504.76,26.789706,0.374406,62.404371
